In [ ]:
# important main libraries
import cvxopt
import numpy as np
from utils.utils import create_dataset, plot_contour

In [ ]:
# kernel class

class Kernel:
    """
    A class to encapsulate various kernel functions for the SVM.

    The class provides static methods for linear, polynomial, and Gaussian (RBF) kernels.
    These methods are used to compute the similarity between data points in the feature space,
    potentially transforming them into a higher-dimensional space for non-linear classification.

    Methods:
    - linear(x, z): Computes the linear kernel between two vectors.
    - polynomial(x, z, p=5): Computes the polynomial kernel between two vectors with degree p.
    - gaussian(x, z, sigma=0.1): Computes the Gaussian (RBF) kernel between two vectors with width sigma.
    """

    @staticmethod
    def linear(x, z):
        """Computes the linear kernel between two vectors."""
        return np.dot(x, z.T)

    @staticmethod
    def polynomial(x, z, p=5):
        """Computes the polynomial kernel between two vectors with degree p."""
        return (1 + np.dot(x, z.T)) ** p

    @staticmethod
    def gaussian(x, z,
                 sigma=0.1,
                 ):
        """Computes the Gaussian (RBF) kernel between two vectors with width sigma."""
       
        return np.exp(-np.linalg.norm(x - z, axis=1) ** 2 / (2 * (sigma ** 2)))

In [ ]:
# svm class
class SVM():
    """
    Support Vector Machine (SVM) class for binary classification.

    This implementation of SVM supports different kernel functions, including linear,
    polynomial, and Gaussian (RBF), and solves the convex optimization problem using
    the cvxopt library.

    Parameters:
    - kernel (function): Kernel function to use for transforming the data. Default is Gaussian.
    - C (float): Regularization parameter. The strength of the regularization is inversely
      proportional to C. Must be strictly positive. The penalty is a squared l2 penalty.

    Attributes:
    - X (numpy.ndarray): The input features used for training.
    - y (numpy.ndarray): The target values used for training.
    - K (numpy.ndarray): The kernel matrix computed from the input features.
    - alphas (numpy.ndarray): The Lagrange multipliers for each data point in the training set.
    - w (numpy.ndarray): The weight vector computed using the support vectors and their corresponding alphas.
    - b (float): The bias term in the decision function.

    Methods:
    - fit(X, y): Trains the SVM model using the provided data.
    - predict(X): Predicts the class labels for the given input features.
    - get_parameters(alphas): Identifies the support vectors, calculates the weight vector w, and computes the bias term b.
    """
    def __init__(self, kernel = Kernel.gaussian, C=1):
        """
        Initializes the SVM model with the specified kernel function and regularization parameter C.
        """
        self.kernel = kernel
        self.C = C
        
    def compute_kernel_matrix(self,X):
        """
        Computes the kernel matrix for the input data using the specified kernel function.

        Parameters:
        - X (numpy.ndarray): Training data, a 2D array of shape (n_samples, n_features).

        Returns:
        - K (numpy.ndarray): The kernel matrix, a 2D array of shape (n_samples, n_samples).
        """
        m, n = X.shape
        m = X.shape[0]  # Number of samples
        
        K = np.zeros((m, m))  # Initialize the kernel matrix with zeros
                    
        # Calculate Kernel
        self.K = np.zeros((m, m))
        for i in range(m):
            self.K[i, :] = self.kernel(X[i, np.newaxis], self.X)

        return K
        
    def solve_svm_qp_problem(self, K,X,y):
        """
        Solves the quadratic programming (QP) problem at the heart of the training process for a Support Vector Machine (SVM).

        The QP problem is formulated to determine the SVM's decision function that best separates the classes in the dataset.

        The standard form of the optimization problem for the softmax margin SVM can be expressed as:

        Maximize with respect to alpha:
            sum(alpha_i) - 1/2 * alpha^T * H * alpha
        Subject to:
            0 <= alpha_i <= C
            sum(alpha_i * y^(i)) = 0

        Which can also be written as:

        Minimize with respect to alpha:
            1/2 * alpha^T * H * alpha - 1^T * alpha
        Subject to:
            -alpha_i <= 0
            alpha_i <= C
            y^T * alpha = 0

        The objective function to minimize is:
            1/2 * alpha^T * P * alpha - q^T * alpha

        Subject to constraints:
            G * alpha <= h
            A * alpha = b

        where P is a matrix derived from the kernel function applied to the training data, q is a vector where each element is -1, 
        G and h enforce the box constraints on the Lagrange multipliers alpha_i ensuring 0 <= alpha_i <= C, and A and b ensure 
        the sum of the product of Lagrange multipliers and their corresponding labels is zero, satisfying the KKT conditions for optimality.
        """
        m, n = X.shape
        m = X.shape[0]  # Number of samples
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(-np.ones((m,1)))
        G = cvxopt.matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
        h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = cvxopt.matrix(y,(1,m),'d')
        b = cvxopt.matrix(np.zeros(1))
        
        cvxopt.solvers.options['show_progress'] = False
        
        solver = cvxopt.solvers.qp(P, q, G, h, A, b)
        return solver
        
    def fit(self, X,y):
        """
        Fits the SVM model to the training data.

        Parameters:
        - X (numpy.ndarray): Training data, a 2D array of shape (n_samples, n_features).
        - y (numpy.ndarray): Target values, a 1D array of shape (n_samples,).

        The method sets up the quadratic programming problem for the dual form of the SVM optimization,
        solves it using cvxopt, and determines the Lagrange multipliers (alphas). It then calculates the
        kernel matrix K using the chosen kernel function.
        """
        self.X = X
        self.y = y
        
        # Calculate the Kernel matrix using the compute_kernel_matrix method
        self.K = self.compute_kernel_matrix(X)
            
        # Solve the QP problem using the kernel matrix K and target values y
        sol = self.solve_svm_qp_problem(self.K,X, y)
        self.alphas = np.array(sol['x'])
        
    def predict(self,X):
        """
        Predicts the class labels for the provided input features.

        Parameters:
        - X (numpy.ndarray): Input features, a 2D array of shape (n_samples, n_features).

        Returns:
        - y_predict (numpy.ndarray): Predicted class labels, a 1D array of shape (n_samples,).
        The prediction is made based on the sign of the decision function: w \cdot x + b.
        """

        y_predict = np.zeros((X.shape[0]))
        sv = self.get_parameters(self.alphas)
        
        for i in range(X.shape[0]):
            y_predict[i] = np.sum(
                self.alphas[sv] * self.y[sv,np.newaxis] *
                self.kernel(X[i], self.X[sv])[:, np.newaxis]
            )
        
        return np.sign(y_predict + self.b)
    
    def get_parameters(self, alphas):
        """
        Identifies support vectors, calculates the weight vector w, and computes the bias b
        using the support vectors and their corresponding Lagrange multipliers (alphas).

        Parameters:
        - alphas (numpy.ndarray): Lagrange multipliers for each data point in the training set.

        Returns:
        - sv (numpy.ndarray): Indices of the support vectors.

        The support vectors are identified by their corresponding alphas, which are greater than a
        small threshold but less than C. The weight vector w and bias b are then calculated using
        these support vectors and their alphas.
        """
        threshold = 1e-4
        
        sv = ((alphas>threshold) * (alphas <self.C)).flatten()
        
        self.w = np.dot(self.X[sv].T, alphas[sv]*self.y[sv, np.newaxis])
        self.b = np.mean(self.y[sv,np.newaxis] - 
                         self.alphas[sv]*self.y[sv,np.newaxis]*self.K[sv,sv][:,np.newaxis])
        
        return sv